Implemented on top of AzureML.
Reference: https://docs.microsoft.com/en-us/azure/machine-learning/

Pipelines are going to be created inside some Azure ML workspace.
If you already have one, you can get the reference:

In [1]:
# if you already have a workspace created, just run this to initialize workspace config
from azureml.core import Workspace

ws = Workspace.from_config()

get_workspace error using subscription_id=9a2d7383-3c7d-492c-94fc-ba65be408672, resource_group_name=cps-dev-cheng, workspace_name=cheng_offline_dashboard


Found the config file in: C:\dev\reinforcement_learning\batch\aml_config\config.json


ProjectSystemException: Workspace not found.

In case if you do not have, any you can either create it on Azure portal or programmatically:

In [2]:
# if you don't have a workspace already created then run this step
from azureml.core import Workspace
ws = Workspace.create(name = "cheng_offline_dashboard",
                      subscription_id = "9a2d7383-3c7d-492c-94fc-ba65be408672",
                      resource_group = "cps-dev-cheng",
                      create_resource_group = False,
                      location = 'eastus'
                     )
ws.write_config()
ws.get_details()

Deploying AppInsights with name chengoffinsightsjaepylge.
Deployed AppInsights with name chengoffinsightsjaepylge.
Deploying ContainerRegistry with name chengoffacrhsjiuwtg.
Deployed ContainerRegistry with name chengoffacrhsjiuwtg.
Deploying KeyVault with name chengoffkeyvaultdtwicqlo.
Deploying StorageAccount with name chengoffstorageflyerxbb.
Deployed KeyVault with name chengoffkeyvaultdtwicqlo.
Deployed StorageAccount with name chengoffstorageflyerxbb.
Deploying Workspace with name cheng_offline_dashboard.
Deployed Workspace with name cheng_offline_dashboard.
Wrote the config file config.json to: C:\dev\reinforcement_learning\batch\aml_config\config.json


{'id': '/subscriptions/9a2d7383-3c7d-492c-94fc-ba65be408672/resourceGroups/cps-dev-cheng/providers/Microsoft.MachineLearningServices/workspaces/cheng_offline_dashboard',
 'name': 'cheng_offline_dashboard',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': '41b33539-ea35-4ac9-b62c-0702b23cb203',
 'description': '',
 'friendlyName': 'cheng_offline_dashboard',
 'creationTime': '2019-04-01T15:32:10.1676241+00:00',
 'containerRegistry': '/subscriptions/9a2d7383-3c7d-492c-94fc-ba65be408672/resourcegroups/cps-dev-cheng/providers/microsoft.containerregistry/registries/chengoffacrhsjiuwtg',
 'keyVault': '/subscriptions/9a2d7383-3c7d-492c-94fc-ba65be408672/resourcegroups/cps-dev-cheng/providers/microsoft.keyvault/vaults/chengoffkeyvaultdtwicqlo',
 'applicationInsights': '/subscriptions/9a2d7383-3c7d-492c-94fc-ba65be408672/resourcegroups/cps-dev-cheng/providers/microsoft.insights/components/chengoffinsightsjaepylge',
 'identityPrincipalId': '5738bc41-

This is a library of pipeline steps / primitives for creating batch training pipelines from logs produced by personalization service. Also, you can find some recipe pipelines which can be either used as-is or as a reference.

For creating pipeline application context is required:

In [ ]:
ws.datastores # list out all datastores

In [ ]:
import application
from application import context

# set up where to read the log files
ctx = context.Context(
    accountName = 'chengmystorage',
    accountKey = 'OOvXDYUpXqfd9kyipcGyThuVQqh+avvHl36hRcBjeA53FBBtLkx0L0dRJhqpMJV4dXVfYGOWngH9qols20rzPQ==',
    appId = 'large',
    appFolder = 'folder'
)

Now you can create batch training pipeline for vw batch training:

Created pipeline can be either submitted for execution:

Or published to be available via REST endpoint:

Example of REST call:
Reference for better understanding of Azure ML authentication:
https://github.com/Azure/MachineLearningNotebooks/blob/f16bf27e26d6ff7a2cf76e8cbcda4ccc9c878a0c/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb

In [ ]:
from pipelines import extract_pipeline
extractPipeline = extract_pipeline.create_pipeline(ws, ctx, parallel_jobs = 2) # I only parallel running 2 jobs for now for debugging purpose

In [ ]:
from azureml.core import Experiment  
regen_outputs = False
extractExperiment = Experiment(ws, 'test_sweep').submit(
    extractPipeline, 
    pipeline_params = {
        "start_date":'10/13/2018',
        "end_date":'10/13/2018'
    }, 
    regenerate_outputs = regen_outputs
)
print("Extract Pipeline is submitted for execution")
from azureml.widgets import RunDetails
RunDetails(extractExperiment).show()